# Trabajo práctico 5 - Template Matching

**Alumnos:**

- Carol lugones Ignacio (100073)
- Torresetti Lisandro (99846)

## Objetivo

A partir del patrón de la cara de Messi (el cual debe ser seleccionado de entre las imágenes) encontrarlo en cada una de las imágenes en las que Messi aparece. Se pide un algoritmo automático que frente a la lectura sucesiva de estas imágenes (y si quieren, pueden probar con imágenes que encuentren ustedes) devuelva la posición de la cara de Messi dentro de la imagen y el nivel de confianza con el que fue hallada (deben definir también el nivel de confianza propuesto).

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
%matplotlib inline

In [ ]:
def plotter(image, title = '', imgSize = (18,9), grayScale = False, step = 25): #Funcion auxiliar para realizar los graficos
    plt.figure(figsize=imgSize)
    plt.title(title, fontsize = 16, fontweight = "bold")
    plt.imshow(image) if not grayScale else plt.imshow(image, cmap='gray', vmin=0, vmax=255)
    plt.yticks(np.arange(0, len(image), step))
    plt.xticks(np.arange(0, len(image[0]), step))
    plt.show()  

In [ ]:
def gaussian_pyramid(img, num_levels):
    lower = img.copy()
    gaussian_pyr = [lower]
    for i in range(num_levels):
        lower = cv.pyrDown(lower)
        gaussian_pyr.append(np.float32(lower))
    return gaussian_pyr

def laplacian_pyramid(gaussian_pyr):
    laplacian_top = gaussian_pyr[-1]
    num_levels = len(gaussian_pyr) - 1
    
    laplacian_pyr = [laplacian_top]
    for i in range(num_levels,0,-1):
        size = (gaussian_pyr[i - 1].shape[1], gaussian_pyr[i - 1].shape[0])
        gaussian_expanded = cv.pyrUp(gaussian_pyr[i], dstsize=size)
        laplacian = np.subtract(gaussian_pyr[i-1], gaussian_expanded)
        laplacian_pyr.append(laplacian)
    return laplacian_pyr
def getLaplacian(img, desiredLevel):
    return laplacian_pyramid(gaussian_pyramid(img, desiredLevel))


Se noto que las imagenes tenian diferente escala y se hizo un analisis de como seria un analisis basico de las imagenes sin igualarles el tamaño. Se descubrio que no encontraba nunca al personaje buscado, entonces se obliga a que todas las imagenes tengan el mismo tamaño asi llevandolas al tamaño de donde sale el template.

In [ ]:
level = 3
#Cargo el padron de imagenes a usar cuando se trata de encontrar a Messi
imgsGray = []
imgsRgb = []
#imgTemplate = cv.cvtColor(cv.imread("./Fotos/Template.jpg"), cv.COLOR_BGR2RGB)
pathOfTemplate = "./Fotos/Messi_1.jpg"
imgTemplate = gaussian_pyramid(cv.imread("./Fotos/Template.jpg", 0), level)[-1]
paths = glob("./Fotos/Messi*")
paths.remove("./Fotos/Messi_1.jpg")
#img = cv.imread(pathOfTemplate)
#img = getLaplacian(img, level)[-1]
#x,y = len(img[0]), len(img)
#imgsRgb.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
#imgsRgb.append(img)
#imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
for pathImg in glob("./Fotos/Messi*"):
    img = cv.imread(pathImg)
    #imgsRgb.append(cv.cvtColor(img, cv.COLOR_BGR2RGB))
    imgsRgb.append(img)
    imgsGray.append(cv.cvtColor(img, cv.COLOR_BGR2GRAY))
    print(f"Nombre: {pathImg}, cantidad en y: {len(img)}, cantidad en x: {len(img[0])}")

In [ ]:
def pyrUp(lowerPoint, originalImg, levelsDone = level):
    laplacian = getLaplacian(originalImg, levelsDone)
    actualCompression = laplacian[1] + cv.pyrUp(lowerPoint)
    for i in range(2, len(laplacian)):
        plotter(actualCompression)
        actualCompression = cv.pyrUp(actualCompression) + laplacian[i]
    return actualCompression
        

In [ ]:
w,h,top_left

In [ ]:
#Se corre el codigo de la practica para ver como lo muestra, saltear celda si no interesa el trabajo de verlo
#Si se quiere ver los graficos descomentar el plot del final, no recomendado porque es bastante feo
modifier = level * 2
w, h = imgTemplate.shape[::-1]
w,h = int(w * modifier), int(h*modifier)
# Los 6 métodos posibles para comparación:
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']

for meth in methods:
    for img_rgb, img_gray in zip(imgsRgb, imgsGray):
        # Hago una copia de la imagen porque ciclo a ciclo le dibujo rectángulos
        img_salida = img_rgb.copy()
        img_gray = gaussian_pyramid(img_gray, level)[-1]
        other = gaussian_pyramid(img_salida, level)[-1]
        method = eval(meth)

        # Aplicamos la coincidencia de patrones
        #--------------------------------------
        res = cv.matchTemplate(img_gray,imgTemplate,method)

        # Encontramos los valores máximos y mínimos
        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)

        # Si el método es TM_SQDIFF o TM_SQDIFF_NORMED, tomamos el mínimo
        if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
            top_left = min_loc
        else:
            top_left = max_loc
        # Marcamos el lugar donde lo haya encontrado
        #----------------------------------------
        bottom_right = ((top_left[0] + int(w / modifier)), (top_left[1] + int(h / modifier)))
        cv.rectangle(other,top_left, bottom_right, 255)
        top_left = int(top_left[0] * modifier), int(top_left[1] * modifier)
        bottom_right = ((top_left[0] + w), (top_left[1] + h))
        cv.rectangle(img_salida,top_left, bottom_right, 255, 2)
        ##Reescalamos:
        # Graficamos el procesamiento y la salida
        #----------------------------------------
        plt.figure()

        # Resultado de coincidencia
        plt.subplot(121),plt.imshow(res,cmap = 'gray')
        plt.title('Matching Result'), plt.xticks([]), plt.yticks([])

        # Imagen original con recuadros
        plt.subplot(121),plt.imshow(img_salida)
        plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
        
        # Imagen original con recuadros
        plt.subplot(122),plt.imshow(other)
        plt.title('Original'), plt.xticks([]), plt.yticks([])


        plt.suptitle(meth)
        plt.show()

In [ ]:
def rectangleBestPosibility(img_rgb, img_gray, method):
    res = cv.matchTemplate(img_gray,template,method)
    threshold = 1
    loc = np.where( res >= threshold)
    while len(loc[0]) == 0:
        threshold -= 0.01
        loc = np.where( res >= threshold)
    for pt in zip(*loc[::-1]):
        cv.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)

    #cv.imshow("pirulito",img_rgb)
    print(f"selected threshold is: {threshold}")
    plotter(img_rgb)

In [ ]:
img_rgb = imgsRgb[0]
img_gray = imgsGray[0]
template = imgTemplate

w, h = template.shape[::-1]
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']
# Fijamos un umbral de "coincidencia"
#threshold = 0.95
for meth in methods:
    method = eval(meth)
    rectangleBestPosibility(img_rgb, img_gray, method)

In [ ]:
laplacian = getLaplacian(img, 4)

In [ ]:
len(laplacian[-1]) / len(laplacian[-2])

# Reconocimiento de caras

In [ ]:
#Vuelvo a cargar todas las imagenes
img_fnames = glob('./Fotos/*')
template = None
imgs = []

for name in img_fnames:
    img = cv.imread(name)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    if 'Messi' in name:
        imgs.append(img)
    else:
        template = img
    
    print('Image name: {}, Width = {}, Height = {}'.format(name, img.shape[0], img.shape[1]))


In [ ]:
#Pasamos todas las imagenes al mismo shape de la que se extrajo el template, o sea Messi_1 
for img in imgs:
    plotter(img)

In [ ]:
plotter(template)

In [ ]:
template.shape[:-1]

In [ ]:
w, h = template.shape[:-1]
# Los 6 métodos posibles para comparación:
methods = ['cv.TM_CCOEFF', 'cv.TM_CCOEFF_NORMED', 'cv.TM_CCORR',
            'cv.TM_CCORR_NORMED', 'cv.TM_SQDIFF', 'cv.TM_SQDIFF_NORMED']

img_rgb = imgs[5]
img_gray = cv.cvtColor(img_rgb, cv.COLOR_RGB2GRAY)
template = cv.cvtColor(template, cv.COLOR_RGB2GRAY)

for meth in methods:
    # Hago una copia de la imagen porque ciclo a ciclo le dibujo rectángulos
    img_salida = img_rgb.copy()
    
    method = eval(meth)
    
    # Aplicamos la coincidencia de patrones
    #--------------------------------------
    res = cv.matchTemplate(img_gray,template,method)
    
    # Encontramos los valores máximos y mínimos
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    
    # Si el método es TM_SQDIFF o TM_SQDIFF_NORMED, tomamos el mínimo
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
    
    # Marcamos el lugar donde lo haya encontrado
    #----------------------------------------
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv.rectangle(img_salida,top_left, bottom_right, 255, 2)
    
    # Graficamos el procesamiento y la salida
    #----------------------------------------
    plt.figure()
    
    # Resultado de coincidencia
    plt.subplot(121),plt.imshow(res,cmap = 'gray')
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])
    
    # Imagen original con recuadros
    plt.subplot(122),plt.imshow(img_salida)
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
    
    plt.suptitle(meth)
    plt.show()

In [ ]:
import cv2

# Load the cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
# Read the input image
imgTest = imgs[5]
# Convert into grayscale
gray = cv2.cvtColor(imgTest, cv2.COLOR_RGB2GRAY)
# Detect faces
faces = face_cascade.detectMultiScale(gray, 1.1, 4)
# Draw rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(imgTest, (x, y), (x+w, y+h), (255, 0, 0), 2)
# Display the output
plotter(imgTest)


In [ ]:
faces